In [ ]:
import mod_sql
import pandas as pd

In [ ]:
# mods_sql을 통해서 Database 클래스를 생성
db = mod_sql.Database('localhost', 'root', 'ubion7', 3306)

In [ ]:
sql = """
    select 
    `Country` as '국가', 
    `Units Sold` as '판매개수', 
    `Unit Price` as '가격', 
    `Unit Cost` as '원가', 
    (`Units Sold` * `Unit Price`) as '총수익', 
    (`Units Sold` * `Unit Cost`) as '총비용', 
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from 
    `Sales Records`
"""
db.sql_query(sql)


In [ ]:
df = pd.read_csv("../csv/Sales Records.csv")

In [ ]:
df.head(4)

In [ ]:
df = df[['Country','Units Sold','Unit Price','Unit Cost']]

In [ ]:
df.columns = ['국가','판매개수','가격','원가']

In [ ]:
df['총수익'] = df['판매개수'] * df['가격']
df['총비용'] = df['판매개수'] * df['원가']
df['총이윤'] = df['총수익'] - df['총비용'] 

# sql에서 반올림

In [ ]:
# 백쿼드로 넣어야함
sql = """
    select
    `Total Cost`,
    round(`Total Cost`, 1)
    from
    `Sales Records`
"""

db.sql_query(sql)

# sql 그룹화
1. Country, Item Type을 기준으로 그룹화
2. Unit Sold의 평균 , 합계 생성
3. Country를 기준으로 오름차순 정리

In [ ]:
sql = """ 
    select
    `Country`,
    `Item Type`,
    AVG(`Units Sold`)as AVG,
    SUM(`Units Sold`)as SUM
    from
    `Sales Records`
    group by
    `Country`,
    `Item Type`
    order by
    `Country`
    ASC
""" 

db.sql_query(sql)

In [ ]:
df = pd.read_csv("../csv/Sales Records.csv")

In [ ]:
df2 = df[['Country','Item Type','Units Sold']]

In [ ]:
# 그룹바이 함수 -> 괄호안의 인자값을 기준으로 값을 묶어준다
df2.groupby(['Country','Item Type']).agg(['mean','sum'])['Units Sold']

### dept table에서 부서의 이름(dname)이 sales인 소속 사원의 정보 출력
1. dept 테이블에서 부서이름이 sales인 deptno를 출력
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 출력

In [ ]:
db = mod_sql.Database('localhost', 'root', 'ubion7', 3306)

In [ ]:
# SQL에서는 where을 써서 조건식을 써주기
sql = """
    select 
    `DEPTNO` 
    from 
    `dept`
    where
    `dname`="sales"
"""
db.sql_query(sql)


In [ ]:
# 백쿼트와 쉼표 잘 써주기
sql = """ 
    select
    *
    from
    `emp` 
    where 
    `deptno` = 30
"""

db.sql_query(sql)

In [ ]:
# 변수는  백쿼트가 아닌 큰따옴표나 따옴표로 써야함 , *는 컬럼을 전체 선택 한다는 표현
sql = """ 
    select
    *
    from
    `emp`
    where
    `deptno` = (
        select
        `deptno`
        from
        `dept`
        where
        `dname` = "research"
    )
"""

db.sql_query(sql)

In [ ]:
sql = """ 
    select
    `deptno`
    from
    `dept`
    where
    `dname` in ("sales", "research")
"""

db.sql_query(sql)

In [ ]:
sql = """ 
    select `deptno` from `dept` where `dname` in ("sales", "research")
"""
db.sql_query(sql)

In [ ]:
sql = """ 
    select * from `emp` where `deptno` in
    (select `deptno` from `dept` where `dname` in ("sales", "research"))
"""
db.sql_query(sql)

In [ ]:
dept = pd.read_csv("../csv/dept.csv")
emp = pd.read_csv("../csv/emp.csv")

In [ ]:
flag = dept.loc[(dept['DNAME'].isin(['RESEARCH','SALES'])),'DEPTNO'].tolist()

In [ ]:
emp.loc[emp["DEPTNO"].isin(flag)].reset_index(drop=True)

1. emp 데이터프레임에서 JOB컬럼의 데이터를 확인하여 새로운 파생변수 생성
2. job_code SALESMAN이면 1 MANAGER면 2 나머지는 3으로 하여 새로운 파생변수에 대입

In [ ]:
# 파생변수 생성
emp["JOB_CODE"]=""

In [ ]:
for i in emp.index:
    if emp.loc[i,'JOB'] == "SALESMAN":
        emp.loc[i,'JOB_CODE'] == 1
    elif emp.loc[i,'JOB'] == "MANGER":
        emp.loc[i,'JOB_CODE'] == 2
    # else:
    #     emp.loc[i,'JOB_CODE'] == 3
emp                

In [ ]:
# 반복문 사용해서 코드 보여주기
emp['JOB_CODE'] = ""
job_list = ['SALESMAN','MANAGER']
for i in job_list:
    emp['JOB_CODE'] = 3
    if i == 'SALESMAN':
        emp.loc[emp['JOB'] == i, 'JOB_CODE'] = 1
    elif i == 'MANAGER':
        emp.loc[emp['JOB'] == i, 'JOB_CODE'] = 2
emp           

In [ ]:
emp["JOB_CODE"] = emp['JOB'].apply(lambda x : 1 if(x == 'SALESMAN')else(
    2 if (x== "MANAGER")else 3
))

In [ ]:
def change(x):
    if x == "SALESMAN":
        result = 1
    elif x == "MANAGER":
        result = 2
    else:
        result = 3
    return result

emp['JOB_CODE'] = emp['JOB'].apply(change)                 

In [ ]:
# case -> if문 과 비슷.
sql = """ 
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    CASE
       WHEN `JOB` = "SALESMAN"
       THEN 1
       WHEN `JOB` = "MANAGER"
       THEN 2
       ELSE 3
    END as `JOB_CODE`
    from
    emp
"""

db.sql_query(sql)

In [ ]:
# 조건식 다음값 다음값이 거짓 -> 3
sql = """ 
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
        if(
            `JOB` = "SALESMAN",
             1,
            if(
               `JOB` = "MANAGER",
               2,
               3
            ) 
        ) as `JOB_CODE`
        from
        emp    
"""

db.sql_query(sql)

1. emp 데이터에서 파생변수 H/L 생성
2. SAL이 2000이상이면 High, 아니면 LOW로 데이터를 삽입

In [ ]:
for i in emp.index:
    if emp.loc[i,"SAL"] >= 2000:
        emp.loc[i,"H/L"] = "High"
    else:
        emp.loc[i,"H/L"] = "LOW"
emp

In [ ]:
emp["H/L"] = ""

In [ ]:
# apply는 출력하면 시리즈 형태의 조건을 넣기 때문에 시리즈 형태로 출력이 된다.
emp["H/L"] = emp['SAL'].apply(lambda x : "High" if( x >= 2000) else "LOW") 

In [ ]:
emp.head(5)

In [ ]:
def change(x):
    if x>= 2000:
        result = "High"
    else:
        result = "LOW"
    return result

emp['SAL'].apply(change)        

In [ ]:
# *하면 전체칼렴 출력, 컬럼명 적으면 특정컬럼 출력
sql = """ 
    select
    *,
    CASE
       WHEN `SAL` >= 2000
       THEN "High"
       ELSE "LOW"
    END as `H/L`
    from
    emp
"""

db.sql_query(sql)

In [ ]:
sql = """ 
    select
    `EMPNO`,
    `ENAME`,
    `SAL`,
    if(
      `SAL` >= 2000,
      'High',
      'LOW'  
    ) as 'H/L' 
    from
    emp
"""

db.sql_query(sql)

# join
1. emp와 dept table을 결합

In [ ]:
# 기준이 되는 파일에 따라서 널값의 경우 나오는게 달라짐.
sql = """ 
    select
    emp.EMPNO,
    emp.ENAME,
    dept.DEPTNO,
    dept.DNAME,
    dept.LOC
    from
    emp
    inner join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

db.sql_query(sql)

In [ ]:
dept = pd.read_csv("../csv/dept.csv")
emp = pd.read_csv("../csv/emp.csv")

In [ ]:
pd.merge(emp,dept,on = "DEPTNO",how = "right")

### 문제
 - 부서의 위치가 'NEW YORK', 'DALLAS'가 아닌 소속 사원의 정보 출력

In [ ]:
sql = """ 
    select DEPTNO from dept
    where
    LOC not in ('NEW YORK','DALLAS')
"""
db.sql_query(sql)

In [ ]:
sql = """ 
    select DEPTNO from dept
    where
    LOC != 'NEW YORK' AND LOC != 'DALLAS'
"""
db.sql_query(sql)

In [ ]:
# 조건을 찾아서 빼는 방법
sql = """ select * from emp where DEPTNO = 30 OR DEPTNO = 40"""
db.sql_query(sql)

In [ ]:
sql = """ select * from emp where DEPTNO IN (30,40)"""
db.sql_query(sql)

In [ ]:
sql = """ select * from emp where DEPTNO in(
     select DEPTNO from dept where LOC not in ('NEW TORK','DALLAS')
)"""
db.sql_query(sql)

In [ ]:
# 조인해서 하는 방법
sql = """ 
    select * from emp left join dept on emp.DEPTNO = dept.DEPTNO
    where LOC not in('NEW YORK' , 'DALLAS')
"""

db.sql_query(sql)

# 테이블의 결합
1. 유니언 결합(단순하게 행을 결합)
2. 조인 결합

In [ ]:
sql = """ 
    select * from tran_1
    union
    select * from tran_2
"""

db.sql_query(sql)

In [ ]:
sql = """ 
    select * from tran_d_1
    union
    select * from tran_d_2
"""
db.sql_query(sql)

In [ ]:
sql = """ 
    select * from 
    tran_1 left join tran_d_1 
    on tran_1.transaction_id = tran_d_1.transaction_id
"""

db.sql_query(sql)

In [ ]:
sql = """ 
    select * from
    (select * from tran_1
    union
    select * from tran_2) as tran
    left join
    (select * from tran_d_1
    union
    select * from tran_d_2) as tran_d
    on tran.transaction_id = tran_d.transaction_id
"""
db.sql_query(sql)

In [ ]:
tran_1 = pd.read_csv("../csv/tran_1.csv")
tran_2 = pd.read_csv("../csv/tran_2.csv")
tran_d_1 = pd.read_csv("../csv/tran_d_1.csv")
tran_d_2 = pd.read_csv("../csv/tran_d_2.csv")

In [ ]:
df1 = pd.concat([tran_1,tran_2],axis=0)
df2 = pd.concat([tran_d_1,tran_d_2],axis=0)
pd.merge(df1,df2,on = "transaction_id",how = "left")

### 문자열이 포함되어 있는 경우 출력

In [ ]:
# %%s -> s로 시작하는말 찾기 %%s%% -> s로 시작하거나 끝나는말  
sql = """ 
   select * from emp
   where ENAME like'%%S%%'
"""

db.sql_query(sql)

1. pandas 이용하여 emp.csv에서 ENAME이 S로 시작하는 데이터를 출력
2. S로 끝나는 데이터를 출력
3. S가 포함되어 있는 데이터를 출력

In [ ]:
emp = pd.read_csv("../csv/emp.csv")

In [ ]:
# emp[emp['ENAME'].str.startswith('S')]
# emp[emp['ENAME'].str.endswith('S')]
# emp[emp['ENAME'].str.contains("S")]

# 인덱스를 정할때 loc를 꼬박꼬박 잘 써야 좋은 결과를 낼수있다.

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x: x.startswith("S"))]

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x: x.endswith("S"))]

In [ ]:
emp.loc[emp['ENAME'].str.contains('S')]

In [ ]:
# 방법1. for
data = pd.DataFrame()
for i in emp.index:
    if emp.loc[i,'ENAME'].startswith('S'):
        data = pd.concat([data,emp.loc[[i]]])

data        

In [ ]:
# 방법2. for
_list = []
for i in emp.index:
    if emp.loc[i,'ENAME'].startswith('S'):
        _list.append(i)

emp.loc[_list]        

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x: x.find('S') == (len(x)-1))]

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x: x.find('S') == 0)]

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x: x.find('S') != -1)]

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x: 'S' in x)]